See also:
* [GML Spec](https://fffff.at/gml/)
* [javascript canvas player example](https://jamiedubs.com/canvasplayer/?random), and [source](https://github.com/jamiew/canvasplayer)

In [ ]:
import json
from pathlib import Path

import load_gml
import matplotlib.pyplot as plt
import numpy as np
import tqdm.notebook as tqdm
from load_gml import Drawing

%load_ext autoreload
%autoreload 2

In [ ]:
# Download (or use cached file)
drawing = load_gml.get_from_blackbook(30083, verbosity=3)

# Draw
def draw_tag(ax, drawing: Drawing):
    for stroke in drawing.strokes:
        ax.plot(stroke[:, 1], stroke[:, 2], 'k-')
        ax.axis('equal')
    ax.set_title(f"Tag {drawing.id} {drawing.raw_dict['gml']['tag']['environment']['up']}")

fig, ax = plt.subplots(figsize=(2, 2))
draw_tag(ax, drawing)

In [ ]:
FOLDER = Path('data/gml')
sorted_fnames = reversed(sorted(FOLDER.glob('*.json'), key=lambda x: int(x.with_suffix('').name)))
katsu_fnames = load_gml.filter_by_application(sorted_fnames)
drawings = map(load_gml.Drawing, katsu_fnames)

# display(next(jsons))

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(15, 10))
for drawing, ax in zip(drawings, reversed(axes.flatten())):
    draw_tag(ax, drawing)
fig.tight_layout()